Data structures in python
* String
* List
* Tuples
* Dictionaries
* Set

*args

**kwargs

In [ ]:
# *args -> positional arguments

def get_prod(*args):
    res = 1
    for arg in args:
        res = res*arg
    return res

get_prod(1, 4, 5, 2, 3, 6)

In [ ]:
# **kwargs

def greet(**kwargs):
    greeting = "Hello"
    if "name" in kwargs:
        greeting += f", {kwargs['name']}"
    if 'age' in  kwargs:
        greeting += f", You are {kwargs['age']} years old"
    if "location" in kwargs:
        greeting += f" from {kwargs['location']}"
    greeting+="!"
    return greeting

print(greet(name='John'))
print(greet(name='John', age=24))
print(greet(name="John", location="New york"))
print(greet(name="John", age=24, location="New york"))

In [ ]:
def check_args(*args):
    return args

def check_kwargs(**kwargs):
    return kwargs

In [ ]:
check_args(1, 2, 3, 4)

In [ ]:
check_kwargs(name="John", age=24, location="New york")

In [ ]:
# Decorators

import time
from datetime import datetime

get_curr_time = lambda : datetime.now().time().strftime("%H:%M:%S")

def timer(func):
    def get_timings(*args, **kwargs):
        start_time = get_curr_time()
        val = func(*args, **kwargs)
        time.sleep(2)
        end_time = get_curr_time()
        print(f"Start time: {start_time} | End time: {end_time} ")
        return val
    return get_timings

@timer
def get_sq(a):
    return a**2

In [ ]:
get_sq(4)

# Chains

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

load_dotenv()
llm = OpenAI()
chat = ChatOpenAI()
set_llm_cache(InMemoryCache())

## LLMChain

In [ ]:
human_template = "Write a film story outline on the topic: {topic}"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat, prompt=chat_prompt)

In [ ]:
result = chain.invoke(input={"topic": "Growth of India"})
result

In [ ]:
chat_prompt.format(**{"topic": "Growth of India"})

In [ ]:
print(result['text'])

In [ ]:
lcel_chain = chat_prompt | chat

lcel_chain.invoke(input={"topic": "Growth of India"})

# Simple Sequential Chain

In [ ]:
story_line_template = "Write a film story outline on the topic: {topic}"
story_line_prompt = ChatPromptTemplate.from_template(template=story_line_template)
story_line_chain = LLMChain(llm=chat, prompt=story_line_prompt)

full_story_template = "Write a short film story on the given story line: {story_line}"
full_story_prompt = ChatPromptTemplate.from_template(template=full_story_template)
full_story_chain = LLMChain(llm=chat, prompt=full_story_prompt)

reviewer_template = "Act as a reviewer of rotten tomatoes and rate the given story: {full_story}"
reviewer_prompt = ChatPromptTemplate.from_template(template=reviewer_template)
reviewer_chain = LLMChain(llm=chat, prompt=reviewer_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain

film_chain = SimpleSequentialChain(
    chains=[story_line_chain, full_story_chain, reviewer_chain],
    verbose=True
)

In [ ]:
result = film_chain.run("Growth of India")

In [ ]:
result

In [ ]:
lcel_film_chain = story_line_chain | full_story_chain | reviewer_chain

lcel_film_chain.invoke(input={"topic": "Growth of India", "story_line": "Growth of India", "full_story": "Growth of India"})

In [ ]:
from langchain.schema.runnable import RunnableLambda

def get_story_line(response):
    return {"story_line": response['text']}

def get_full_story(response):
    return {"full_story": response['text']}

story_line_lcel_chain = story_line_chain | RunnableLambda(get_story_line)
full_story_lcel_chain = full_story_chain | RunnableLambda(get_full_story)

lcel_film_chain = story_line_lcel_chain | full_story_lcel_chain | reviewer_chain

lcel_film_chain.invoke(input={"topic": "Growth of India"})

# Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
story_line_template = "Write a film story outline on the topic: {topic}"
story_line_prompt = ChatPromptTemplate.from_template(template=story_line_template)
story_line_chain = LLMChain(llm=chat, prompt=story_line_prompt, output_key="story_line")

full_story_template = "Write a short film story on the given story line: {story_line}"
full_story_prompt = ChatPromptTemplate.from_template(template=full_story_template)
full_story_chain = LLMChain(llm=chat, prompt=full_story_prompt, output_key="full_story")

reviewer_template = "Act as a reviewer of rotten tomatoes and rate the given story: {full_story}\n and the topic is: {topic}"
reviewer_prompt = ChatPromptTemplate.from_template(template=reviewer_template)
reviewer_chain = LLMChain(llm=chat, prompt=reviewer_prompt, output_key="reviwer_response")

In [ ]:
lcel_chain = story_line_chain | full_story_chain | reviewer_chain

In [ ]:
lcel_chain.invoke(input={"topic": "Growth of India"})

In [ ]:
seq_chain = SequentialChain(
    chains=[story_line_chain, full_story_chain, reviewer_chain],
    input_variables=['topic'],
    output_variables = ['story_line', 'full_story', 'reviwer_response'],
    verbose=True
)

seq_chain.invoke(input={"topic": "Growth of India"})

# LCEL

In [ ]:
from langchain.schema.runnable import RunnableLambda

def square(a):
    return a**2

sq_runnable = RunnableLambda(square)

sq_runnable

In [ ]:
type(sq_runnable)

In [ ]:
sq_runnable.invoke(4)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

runnable_sq_pass = RunnablePassthrough()

In [ ]:
runnable_sq_pass.invoke(5)

In [ ]:
from langchain.schema.runnable import RunnableParallel
from operator import itemgetter


def times2(x):
    return x*2


def times3(y):
    return y*3


# def add(res_dict):
#     return res_dict['a'] + res_dict['b']
def add(res_dict):
    res = 0
    for k, v in res_dict.items():
        res+=v
    return res

runnable_times2 = RunnableLambda(times2)
runnable_times3 = RunnableLambda(times3)
runnable_sum = RunnableLambda(add)

# par_chain = RunnableParallel({"a": itemgetter('x') | runnable_times2, 
#                               "b": itemgetter('y') | runnable_times3})

par_chain = RunnableParallel(
    {"a": itemgetter('x') | runnable_times2, 
     "b": itemgetter('y') | runnable_times3,
     "x": itemgetter('x') | RunnablePassthrough(),
     "y": itemgetter('y') | RunnablePassthrough(),
     })

calc_chain = par_chain | runnable_sum

calc_chain.invoke(input={"x": 2, "y": 3})

In [ ]:
class RunnableLambdaTest:
    def __init__(self, func):
        self.func = func
        
    def __or__(self, other_runnable_obj):
        def chained_func(*args, **kwargs):
            return other_runnable_obj.invoke(self.func(*args, **kwargs))
        return RunnableLambdaTest(chained_func)
    
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
def times2(a):
    return a*2

def times3(b): 
    return b*3

runnable_2 = RunnableLambdaTest(times2)
runnable_3 = RunnableLambdaTest(times3)

# test_chain = runnable_2.__or__(runnable_3)

test_chain = runnable_2 | runnable_3

test_chain.invoke(2)

# LLMRouterChain

In [ ]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

warnings.filterwarnings("ignore")
load_dotenv()
llm = OpenAI()
chat = ChatOpenAI()
set_llm_cache(InMemoryCache())

In [ ]:
# Task types

psy_name = "Psychologist" # Who

psy_description = "Helps with any psychological issues" # What

psy_template = """You are a helpful psychologist who wants to help the user to make them feel better.
You always give 3 simple and most appropriate remedies to the user based on their issue.
The user issue is mentioned in the below triple backticks:
```
{input}
```
""" # How

# ----------------------------------------------

phy_name = "General Physician"

phy_description = "Helps with any General Physical Health issues"

phy_template = """You are a helpful general physician who always suggests home remedies for any issues.
You always suggest 5 most appropriate remedies to the user based on their issue.
The user issue is mentioned in the below triple backticks:
```
{input}
```
"""

In [ ]:
llm_dict = {psy_name: chat, phy_name: chat}

In [ ]:
prompt_info_list = [
    {
        "name": psy_name,
        "description": psy_description,
        "template": psy_template
    }, 
    {
        "name": phy_name,
        "description": phy_description,
        "template": phy_template
    }, 
]

In [ ]:
destination_chains = {}
destination_descriptions = ""

for prompt_info in prompt_info_list:
    name = prompt_info['name']
    template = prompt_info['template']
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = llm_dict[name]
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    
    destination_descriptions += f"{prompt_info['name']}: {prompt_info['description']}\n"

In [ ]:
destination_chains

In [ ]:
print(destination_descriptions)

In [ ]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_descriptions)

In [ ]:
router_prompt = PromptTemplate(template=router_template, input_variables=[], output_parser=RouterOutputParser())

In [ ]:
router_chain = LLMRouterChain.from_llm(llm=chat, prompt=router_prompt)

In [ ]:
from langchain.chains import ConversationChain

default_chain = ConversationChain(llm=chat, output_key='text')

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [ ]:
response = chain.run("I could not sleep last night")
print(response)

In [ ]:
response = chain.run("I couldn't walk properly since I hit my leg to the bed yesterday")
print(response)

In [ ]:
response = chain.run("Taj mahal is beautiful")
print(response)

In [ ]:
response = chain.run("I don't like taj mahal")
print(response)

## MathChain

In [ ]:
!pip install numexpr

### LLMMathChain

In [ ]:
from langchain import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm=chat, verbose=True)

In [ ]:
llm_math_chain.run("What is 23 multiplied by 4 and subtracted by 18?")

### PALChain

In [ ]:
from langchain_experimental.pal_chain import PALChain

In [ ]:
pal_chain = PALChain.from_math_prompt(llm=chat, verbose=True)

In [ ]:
pal_chain.run("A train travels at a speed of 60km/h and covers a certain distance in 3 hours. What is the distance covered by the train?")

In [ ]:
# Task

from langchain.chains import MultiRetrievalQAChain

chain = MultiRetrievalQAChain.from_retrievers(
    llm=chat,
    retriever_infos, 
    default_chain
)